In [ ]:
import os

YA_TOKEN=os.getenv('YA_TOKEN')

# source
SOURCE_DISK_ROOT = '/Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/'
SOURCE_PUBLIC_URL = "https://disk.yandex.ru/d/INOsQEAaSA8QcA"

# destination
DEST_DISK_ROOT = 'Загрузки/MRT_PNGs'
ARTIFACTS_DIR = os.path.join(os.getcwd(), "artifacts")
DEST_LOCAL_ROOT = os.path.join(ARTIFACTS_DIR, DEST_DISK_ROOT)


# script settings
SKIP_IF_PNG_EXISTS = True
UPLOAD_METADATA_EVERY = None # upload only at the end
MAX_FILES_FOR_TEST = 50 # set None for full run

# Image conversion settings
CLIP_PERCENTILES = (1, 99)
OUTPUT_MODE = 'L'
PNG_COMPRESS_LEVEL = 6

## 1) Connect to Yandex Disk

In [84]:
import io
import os
import csv
import posixpath
import datetime as _dt
from dataclasses import dataclass
from typing import Iterator, Optional, Dict, Any, List, Tuple

import yadisk
import numpy as np
import pandas as pd
from tqdm import tqdm

import pydicom
from pydicom.errors import InvalidDicomError
from PIL import Image

y_auth = yadisk.Client(token=YA_TOKEN)
y_public = yadisk.Client()

if y_auth is None:
    raise ValueError('set YA_TOKEN in env variable to upload PNGs to Disk')

print('Token valid:', y_auth.check_token())
info = y_auth.get_disk_info()
print('Disk info (short):', {'total_space': info.total_space, 'used_space': info.used_space})

Token valid: True
Disk info (short): {'total_space': 1104880336896, 'used_space': 80572290684}


In [85]:
@dataclass
class SourceFile:
    src_mode: str # disk or public
    src_path: str # for disk, absolute disk path, for public, rel path within public folder
    rel_path: str # rel path under source root
    name: str

def parse_labels_from_patient_folder(patient_folder: str) -> Dict[str, Any]:
    return {
        'patient_folder': patient_folder,
        'has_bp': ('БП' in patient_folder),
        'has_et': ('ЭТ' in patient_folder),
    }

def ensure_remote_dir(client: yadisk.Client, remote_dir: str):
    remote_dir = remote_dir.rstrip('/') or '/'
    if remote_dir == '/':
        return
    
    try:
        client.makedirs(remote_dir)
        return
    except Exception:
        pass

    parts = [p for p in remote_dir.split('/') if p]
    cur = ''
    for p in parts:
        cur = cur + '/' + p
        try:
            if not client.exists(cur):
                client.mkdir(cur)
        except yadisk.exceptions.PathExistsError:
            print('cannot make directories for', remote_dir)
            pass


def iter_disk_files(local_root: str) -> Iterator[SourceFile]:
    """Yield all files under a local folder, preserving rel_path."""
    root_norm = os.path.abspath(local_root)
    for dirpath, _, filenames in os.walk(root_norm):
        for filename in filenames:
            abs_path = os.path.join(dirpath, filename)
            rel_path = os.path.relpath(abs_path, root_norm)
            yield SourceFile(src_mode="disk", src_path=abs_path, rel_path=rel_path, name=filename)

def iter_public_files(client: yadisk.Client, public_url: str) -> Iterator[SourceFile]:
    """Yield all files under a public folder link, preserving rel_path."""
    stack = ['']
    while stack:
        cur_rel = stack.pop()
        for item in client.public_listdir(public_url, path=cur_rel or None):
            child_rel = posixpath.join(cur_rel, item.name) if cur_rel else item.name
            if item.type == "dir":
                stack.append(child_rel)
            else:
                yield SourceFile(src_mode="public", src_path=child_rel, rel_path=child_rel, name=item.name)

def iter_source_files() -> Iterator[SourceFile]:
    if SOURCE_DISK_ROOT:
        yield from iter_disk_files(SOURCE_DISK_ROOT)
    elif SOURCE_PUBLIC_URL:
        yield from iter_public_files(y_public, SOURCE_PUBLIC_URL)
    else:
        raise ValueError("Set either SOURCE_DISK_ROOT or SOURCE_PUBLIC_URL")

## 2) DICOM → PNG conversion (2D)

Strategy:
- Attempt to read the file as DICOM.
- If it contains `PixelData`, convert to a normalized 8-bit grayscale PNG.
- If it is multi-frame, output one PNG per frame with a suffix.

In [ ]:
def _get_first_number(x: Any) -> Optional[float]:
    if x is None:
        return None
    try:
        if isinstance(x, (list, tuple)) and len(x) > 0:
            return float(x[0])
    except Exception:
        pass
    try:
        return float(x)
    except Exception:
        return None

def dicom_to_png_bytes(dicom_bytes: bytes) -> Tuple[List[Tuple[str, bytes]], Dict[str, Any]]:
    """Returns ([(suffix, png_bytes), ...], meta). suffix is '' or '_f000' etc."""
    bio = io.BytesIO(dicom_bytes)
    ds = pydicom.dcmread(bio, force=True)

    meta: Dict[str, Any] = {}
    meta["study_instance_uid"] = getattr(ds, "StudyInstanceUID", None)
    meta["series_instance_uid"] = getattr(ds, "SeriesInstanceUID", None)
    meta["sop_instance_uid"] = getattr(ds, "SOPInstanceUID", None)
    meta["modality"] = getattr(ds, "Modality", None)
    meta["series_description"] = getattr(ds, "SeriesDescription", None)
    meta["instance_number"] = getattr(ds, "InstanceNumber", None)
    meta["acquisition_number"] = getattr(ds, "AcquisitionNumber", None)

    meta["rows"] = getattr(ds, "Rows", None)
    meta["cols"] = getattr(ds, "Columns", None)
    meta["pixel_spacing"] = list(getattr(ds, "PixelSpacing", [])) if hasattr(ds, "PixelSpacing") else None
    meta["slice_thickness"] = _get_first_number(getattr(ds, "SliceThickness", None))
    meta["slice_location"] = _get_first_number(getattr(ds, "SliceLocation", None))
    meta["image_position_patient"] = list(getattr(ds, "ImagePositionPatient", [])) if hasattr(ds, "ImagePositionPatient") else None

    meta["bits_allocated"] = getattr(ds, "BitsAllocated", None)
    meta["photometric_interpretation"] = getattr(ds, "PhotometricInterpretation", None)
    meta["transfer_syntax_uid"] = getattr(getattr(ds, "file_meta", None), "TransferSyntaxUID", None)

    if not hasattr(ds, "PixelData"):
        raise ValueError("No PixelData in DICOM")

    arr = ds.pixel_array.astype(np.float32)

    slope = _get_first_number(getattr(ds, "RescaleSlope", 1.0)) or 1.0
    intercept = _get_first_number(getattr(ds, "RescaleIntercept", 0.0)) or 0.0
    arr = arr * slope + intercept

    wc = _get_first_number(getattr(ds, "WindowCenter", None))
    ww = _get_first_number(getattr(ds, "WindowWidth", None))

    def normalize_to_u8(x: np.ndarray) -> np.ndarray:
        if wc is not None and ww is not None and ww > 0:
            lo = wc - ww / 2.0
            hi = wc + ww / 2.0
            x = np.clip(x, lo, hi)
        else:
            p_lo, p_hi = np.percentile(x, CLIP_PERCENTILES)
            if p_hi <= p_lo:
                p_lo, p_hi = float(np.min(x)), float(np.max(x))
            x = np.clip(x, p_lo, p_hi)
        x = (x - x.min()) / (x.max() - x.min() + 1e-6)
        return (x * 255.0).round().astype(np.uint8)

    pngs: List[Tuple[str, bytes]] = []
    if arr.ndim == 2:
        u8 = normalize_to_u8(arr)
        im = Image.fromarray(u8, mode=OUTPUT_MODE)
        out = io.BytesIO()
        im.save(out, format="PNG", compress_level=PNG_COMPRESS_LEVEL)
        pngs.append(("", out.getvalue()))
    elif arr.ndim == 3:
        for i in range(arr.shape[0]):
            u8 = normalize_to_u8(arr[i])
            im = Image.fromarray(u8, mode=OUTPUT_MODE)
            out = io.BytesIO()
            im.save(out, format="PNG", compress_level=PNG_COMPRESS_LEVEL)
            pngs.append((f"_f{i:03d}", out.getvalue()))
        meta["number_of_frames"] = arr.shape[0]
    else:
        raise ValueError(f"Unsupported pixel_array ndim={arr.ndim}")

    return pngs, meta

In [87]:
# Download + upload helpers
def download_source_bytes(sf: SourceFile) -> bytes:
    if sf.src_mode == "disk":
        with open(sf.src_path, "rb") as f:
            return f.read()
    elif sf.src_mode == "public":
        out = io.BytesIO()
        y_public.download_public(SOURCE_PUBLIC_URL, out, path=sf.src_path)
        return out.getvalue()
    else:
        raise ValueError(f"Unsupported src_mode: {sf.src_mode}")

def save_png_locally(png_bytes: bytes, dest_path: str) -> str:
    # dest_path is a posix path under DEST_DISK_ROOT that ends with .png
    rel_dest = posixpath.relpath(dest_path, DEST_DISK_ROOT.rstrip('/'))
    local_path = os.path.join(DEST_LOCAL_ROOT, rel_dest.replace("/", os.sep))
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    with open(local_path, "wb") as f:
        f.write(png_bytes)
    return local_path

def upload_artifacts_via_api() -> None:
    if not os.path.isdir(DEST_LOCAL_ROOT):
        raise RuntimeError(f"Local artifacts dir not found: {DEST_LOCAL_ROOT}")
    ensure_remote_dir(y_auth, DEST_DISK_ROOT)
    for dirpath, _, filenames in os.walk(DEST_LOCAL_ROOT):
        rel_dir = os.path.relpath(dirpath, DEST_LOCAL_ROOT)
        remote_dir = DEST_DISK_ROOT.rstrip('/') if rel_dir == '.' else posixpath.join(DEST_DISK_ROOT.rstrip('/'), rel_dir.replace(os.sep, '/'))
        ensure_remote_dir(y_auth, remote_dir)
        for filename in filenames:
            local_path = os.path.join(dirpath, filename)
            rel_path = os.path.relpath(local_path, DEST_LOCAL_ROOT).replace(os.sep, '/')
            remote_path = posixpath.join(DEST_DISK_ROOT.rstrip('/'), rel_path)
            y_auth.upload(local_path, remote_path, overwrite=True)

def delete_local_artifacts() -> None:
    for dirpath, _, filenames in os.walk(DEST_LOCAL_ROOT):
        for filename in filenames:
            local_path = os.path.join(dirpath, filename)
            os.remove(local_path)

In [88]:
# Metadata CSV writer
os.makedirs(DEST_LOCAL_ROOT, exist_ok=True)

METADATA_LOCAL_PATH = os.path.join(DEST_LOCAL_ROOT, "metadata.csv")
METADATA_REMOTE_PATH = posixpath.join(DEST_DISK_ROOT.rstrip('/'), 'metadata.csv')

CSV_FIELDS = [
    "processed_at",
    "status",
    "error",
    "src_mode",
    "src_rel_path",
    "src_path",
    "dest_png_path",
    "frame_index",
    "patient_folder",
    "has_bp",
    "has_et",
    "study_instance_uid",
    "series_instance_uid",
    "sop_instance_uid",
    "modality",
    "series_description",
    "instance_number",
    "acquisition_number",
    "rows",
    "cols",
    "pixel_spacing",
    "slice_thickness",
    "slice_location",
    "image_position_patient",
    "bits_allocated",
    "photometric_interpretation",
    "transfer_syntax_uid",
    "number_of_frames",
]

def init_metadata_csv(path: str) -> None:
    if os.path.exists(path) and os.path.getsize(path) > 0:
        return
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=CSV_FIELDS)
        w.writeheader()

def append_metadata_row(path: str, row: Dict[str, Any]) -> None:
    with open(path, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=CSV_FIELDS)
        safe_row = {k: row.get(k, None) for k in CSV_FIELDS}
        w.writerow(safe_row)

def upload_metadata_csv() -> None:
    ensure_remote_dir(y_auth, posixpath.dirname(METADATA_REMOTE_PATH))
    y_auth.upload(METADATA_LOCAL_PATH, METADATA_REMOTE_PATH, overwrite=True)

init_metadata_csv(METADATA_LOCAL_PATH)
print("metadata.csv local:", METADATA_LOCAL_PATH)
print("metadata.csv remote:", METADATA_REMOTE_PATH)


metadata.csv local: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/metadata.csv
metadata.csv remote: MPT_AAA_PNGs/metadata.csv


## 3. Main Processing Loop
- walks the source tree
- converts each DICOM file with PixelData
- uploads PNGs
- appends a row to metadata.csv for every produced PNG

In [89]:
def dest_png_path_from_rel(rel_path: str, frame_suffix: str = "") -> str:
    rel_dir = posixpath.dirname(rel_path)
    name = posixpath.basename(rel_path)
    base, ext = os.path.splitext(name)
    out_name = f"{base}{frame_suffix}.png" if ext else f"{name}{frame_suffix}.png"
    if rel_dir:
        return posixpath.join(DEST_DISK_ROOT.rstrip("/"), rel_dir, out_name)
    return posixpath.join(DEST_DISK_ROOT.rstrip("/"), out_name)

def get_patient_folder_from_rel(rel_path: str) -> str:
    return rel_path.split("/", 1)[0] if "/" in rel_path else rel_path

def process_all(max_files: Optional[int] = None):
    ensure_remote_dir(y_auth, DEST_DISK_ROOT)

    processed_rows = 0
    processed_files = 0
    skipped_files = 0
    errors = 0

    for sf in tqdm(iter_source_files(), desc="Scanning/processing"):
        if max_files is not None and processed_files >= max_files:
            break

        name_lower = sf.name.lower()
        #TODO: improve DICOM file detection with DICOMDIR?
        if not (sf.name.startswith('IM') or name_lower.endswith('.dcm') or ('.' not in sf.name)):
            continue

        patient_folder = get_patient_folder_from_rel(sf.rel_path)
        labels = parse_labels_from_patient_folder(patient_folder)

        try:
            if SKIP_IF_PNG_EXISTS:
                expected = dest_png_path_from_rel(sf.rel_path, frame_suffix="")
                if y_auth.exists(expected):
                    skipped_files += 1
                    continue
            
            dicom_bytes = download_source_bytes(sf)
            pngs, dicom_meta = dicom_to_png_bytes(dicom_bytes)
            print(f'Converted DICOM to {len(pngs)} PNG(s):', sf.src_path)

            for frame_i, (suffix, png_bytes) in enumerate(pngs):
                dest_path = dest_png_path_from_rel(sf.rel_path, frame_suffix=suffix)
                fname = save_png_locally(png_bytes, dest_path)
                print('Saved PNG locally:', fname)

                row = {
                    "processed_at": _dt.datetime.utcnow().isoformat() + "Z",
                    "status": "OK",
                    "error": None,
                    "src_mode": sf.src_mode,
                    "src_rel_path": sf.rel_path,
                    "src_path": sf.src_path,
                    "dest_png_path": dest_path,
                    "frame_index": frame_i if len(pngs) > 1 else 0,
                    **labels,
                    **dicom_meta,
                }

                append_metadata_row(METADATA_LOCAL_PATH, row)
                processed_rows += 1

                if UPLOAD_METADATA_EVERY and (processed_rows % UPLOAD_METADATA_EVERY) == 0:
                    upload_metadata_csv()
                    upload_artifacts_via_api()
                    delete_local_artifacts()
            processed_files += 1  

        except InvalidDicomError:
            continue
        except Exception as e:
            errors += 1
            row = {
                "processed_at": _dt.datetime.utcnow().isoformat() + "Z",
                "status": "ERROR",
                "error": repr(e)[:2000],
                "src_mode": sf.src_mode,
                "src_rel_path": sf.rel_path,
                "src_path": sf.src_path,
                "dest_png_path": "",
                "frame_index": "",
                **labels,
            }
            append_metadata_row(METADATA_LOCAL_PATH, row)
        
    upload_metadata_csv()
    upload_artifacts_via_api()
    print('Done.')
    print({
        "processed_files": processed_files,
        "processed_rows": processed_rows,
        "skipped_files": skipped_files,
        "errors": errors,
        "dest_root": DEST_DISK_ROOT,
        "metadata_remote": METADATA_REMOTE_PATH,
    })

In [90]:
process_all(max_files=MAX_FILES_FOR_TEST)

Scanning/processing: 0it [00:00, ?it/s]/var/folders/0_/9m3v107s267fp_fzb5x700140000gn/T/ipykernel_2890/2464272035.py:76: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": _dt.datetime.utcnow().isoformat() + "Z",
Scanning/processing: 2it [00:00,  2.38it/s]/var/folders/0_/9m3v107s267fp_fzb5x700140000gn/T/ipykernel_2890/2464272035.py:50: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": _dt.datetime.utcnow().isoformat() + "Z",
Scanning/processing: 55it [00:01, 56.88it/s]

Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000105
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000105.png
Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000102
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000102.png
Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000103
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000103.png
Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000104
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs

Scanning/processing: 64it [00:04, 12.86it/s]

Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000078
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000078.png
Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000040
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000040.png
Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000082
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000082.png
Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000076
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs

Scanning/processing: 69it [00:06,  8.11it/s]

...

Scanning/processing: 103it [00:22,  2.70it/s]

Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000031
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000031.png


Scanning/processing: 104it [00:23,  4.52it/s]

Converted DICOM to 1 PNG(s): /Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2024/DICOM/PA000000/ST000000/SE000004/IM000036
Saved PNG locally: /Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000036.png


Done.
{'processed_files': 50, 'processed_rows': 50, 'skipped_files': 0, 'errors': 1, 'dest_root': 'MPT_AAA_PNGs', 'metadata_remote': 'MPT_AAA_PNGs/metadata.csv'}
